# DOWNLOADS

In [ ]:
pip install googletrans

FileNotFoundError: ignored

In [ ]:
import nltk
nltk.download('all')
nltk.download('tagset')
nltk.download('punkt')


# Abbreviations - Post tagging

Abbreviation - Meaning

CC- coordinating conjunction 

CD - cardinal digit

DT - determiner
 
EX- existential there 

FW - foreign word 

IN - preposition/subordinating conjunction 

JJ - This NLTK POS Tag is an adjective (large)

JJR - adjective, comparative (larger)

JJS - adjective, superlative (largest)

LS - list market

MD - modal (could, will)

NN - noun ,singular (cat, tree)

NNS - noun plural (desks)

NNP - proper noun,singular (sarah)

NNPS - proper noun, plural (indians or americans),PDT	predeterminer (all, both, half)

POS - possessive ending (parent\ ‘s) , PRP	personal pronoun (hers, herself, him, himself)

PRP$ - possessive pronoun (her, his, mine, my, our )

RB - adverb (occasionally, swiftly)

RBR - adverb ,comparative (greater)

RBS - adverb ,superlative (biggest)

RP - particle (about)

TO - infinite marker (to)

UH - interjection (goodbye)

VB - verb (ask)

VBG - verb gerund (judging)

VBD - verb past tense (pleaded)

VBN - verb past participle (reunified)

VBP - verb , present tense not 3rd person singular(wrap)

VBZ - verb ,present tense with 3rd person singular (bases)

WDT - wh-determiner (that, what)

WP	wh- pronoun (who)

WRB - wh- adverb (how)


# CODE

In [ ]:
from nltk.tag import tnt
from nltk.corpus import indian
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import pos_tag, word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.getcwd()


Loading dataset from drive

In [ ]:
data_folder = os.path.join(os.getcwd(),'/content/drive/MyDrive/input')
print(data_folder)

Loading dataset into a dataframe

In [ ]:
train_text = list()
for root,folders,files in os.walk(data_folder):
  for file in files:
    path=os.path.join(root,file)
    with open(path) as inf:
      txt = ""
      for line in inf:
        if not line.strip():
          continue
        d = line.strip()
        txt += d
      train_text.append(txt)
train_df = pd.DataFrame()
train_df["Text"] = train_text
train_df.head(5)

In [ ]:
print(len(train_df))

In [ ]:
import string
import re
my_punct = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.', '/', ':', ';', '<', '=', '>', '@', '[', '\\', ']', '^', '_', '`', '{', '}', '~', '»', '«', '“', '”',"|"]
for i in range(len(train_df)):
  txt = train_df._get_value(i, 'Text')
  
  punct_pattern = re.compile("[" + re.escape("".join(my_punct)) + "]") 
  train_df.at[i,'Text']=  re.sub(punct_pattern, "", txt)
train_df.head()


In [ ]:
hindi_stopwords=pd.read_csv('/content/final_stopwords.txt', delimiter='\n', header=None)[0].values
hindi_stopwords

In [ ]:
hcount=0
l=[]
for ans in train_df["Text"]:
    tokens=ans.split()
    for token in tokens:
        if token in hindi_stopwords:
            l.append(token)
            hcount+=1
print(hcount)
for i in train_df["Text"]:
  print(i,len(i))
  break
train_df['article'] = train_df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (hindi_stopwords)]))
for i in train_df["article"]:
  print(i,len(i))
  break

Tokenization and Keyword generation(based on regex)

In [ ]:
from nltk.tag import tnt
from nltk.corpus import indian
import nltk
from nltk.tree import Tree

def hindi_model():
    train_data = indian.tagged_sents('hindi.pos')
    tnt_pos_tagger = tnt.TnT()
    tnt_pos_tagger.train(train_data)
    return tnt_pos_tagger


def get_keywords(pos):
    grammar = r"""NP:{<NN.*>}"""
    #grammar = r"""NP:{<DT>?<JJ>*<NN>}"""
    chunkParser = nltk.RegexpParser(grammar)
    chunked = chunkParser.parse(pos)
    continuous_chunk = set()
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.add(named_entity)
                current_chunk = []
            else:
                continue
    return (continuous_chunk)



for text in train_df['article']:
  model = hindi_model()
  print(text)
  print()
  new_tagged = (model.tag(nltk.word_tokenize(text)))
  print(set(new_tagged))
  print()
  print("====KEYWORDS===")
  print(get_keywords(new_tagged))
  print("\n")
  

:Removing Punctuations from the text

In [ ]:
def hindi_model():
    train_data = indian.tagged_sents('hindi.pos')
    tnt_pos_tagger = tnt.TnT()
    tnt_pos_tagger.train(train_data)
    return tnt_pos_tagger

for text in train_df['Text']:
  model = hindi_model()
  new_tagged = (model.tag(nltk.word_tokenize(text)))
  print(set(new_tagged))
  print()
  


In [ ]:
for i in range(0,len(new_tagged)):
  if new_tagged[i][1] == "PREP":
    print(new_tagged[i][0])

In [ ]:
#train_df.to_csv("train_csv.csv")

In [ ]:
from nltk.tag import tnt
from nltk.corpus import indian
from googletrans import Translator
import nltk
import re

In [ ]:
pip install googletrans==4.0.0rc1

Handling the Unknown pos tag words

In [ ]:
translator = Translator()
sentence_id = 0

model_path = "/content/hindi.pos" 

def train_hindi_model(model_path):
    train_data = indian.tagged_sents(model_path)
    tnt_pos_tagger = tnt.TnT()
    tnt_pos_tagger.train(train_data)
    return tnt_pos_tagger


def get_sentId(model_path):
    ids = re.compile('<Sentence\sid=\d+>')
    with open(model_path, "r+") as temp_f:
        content = temp_f.readlines()
        for i in content:
            id_found = (ids.findall(i))
            if id_found:
                id_found = str(id_found).replace("['<Sentence id=", "").replace(">']", "")
                id = int(id_found)
    id = id + 1
    return id


def tag_words(model,text):
    tagged = (model.tag(nltk.word_tokenize(text)))
    return tagged


def handle_UNK(tagged_words, model_path, sentence_id):
    with open(model_path, "r+") as f1:
        result_list = []
        for nep_word, tag in tagged_words:
            if tag == "Unk":
                x = translator.translate(nep_word)
                if x is not None:
                    str1 = str(x)
                    new_str = str1.split()
                    for j in new_str:
                        if re.search('^text=', j, re.I):
                            word = j.replace("text=", ",").replace(",", "")
                            word = str(word)
                            pos = nltk.tag.pos_tag([word])
                            for en_word, tag in pos:
                                result = nep_word + "_" + (tag) + " "
                                result_list.append(result)

            else:
                result = nep_word + "_" + (tag) + " "
                result_list.append(result)

        writing_word = str("\n<Sentence id=") + str(sentence_id) + ">\n"
        output = writing_word + "".join(result_list) + "\n</Sentence>\n</Corpora>"
        for line in f1.readlines():
            f1.write(line.replace("</Corpora>", ""))
        f1.write(output)


sentence_id = (get_sentId(model_path))
model = train_hindi_model(model_path)
for text in train_df['article']:
  tagged_words = tag_words(model,text)
  
  handle_UNK(tagged_words,model_path,sentence_id)

#retrain the model
model = train_hindi_model(model_path)
new_tagged_words =  tag_words(model,text)
print ("=================================New Tagged words=================================\n\n",new_tagged_words,"\n")